# Data Privacy Final Project
### Sam Clark & Josh Childs

For this project, we've decided to compare the accuracy of several normal Convolutional Neural Networks to their counter parts that will use differential privacy. We will be using the MNIST dataset with the tensflow library.  

In [ ]:
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
from ipywidgets import IntProgress
from sklearn.metrics import classification_report

tf.enable_v2_behavior()

## Load MNIST Data

In [17]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


## Preprocess Data

In [19]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

x_train shape: (60000, 28, 28, 1)
Number of images in x_train 60000
Number of images in x_test 10000


## Build Models

In [38]:
model1 = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(28, kernel_size=(3,3), input_shape=input_shape),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation=tf.nn.relu),  
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(10,activation=tf.nn.softmax)
])
model.summary()

models = [model1]

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 28)        280       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 28)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 4732)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               605824    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                1290      
Total params: 607,394
Trainable params: 607,394
Non-trainable params: 0
________________________________________________

## Train & Save

In [44]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

for model in models:
    model.compile(optimizer='adam', 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])

    model.fit(x=x_train,y=y_train, epochs=1, callbacks = [callback])
    model.save("models/" + str(models.index(model) + 1 ))

1875/1875 [==============================] - 21s 11ms/step - loss: 0.0742 - accuracy: 0.9769
INFO:tensorflow:Assets written to: models/1/assets


INFO:tensorflow:Assets written to: models/1/assets


## Evaluate Models

In [42]:
metrics = {}

for model in models:
    metrics["model" + str(models.index(model) + 1)] = model.evaluate(x_test,y_test)
    

313/313 [==============================] - 1s 3ms/step - loss: 0.0637 - accuracy: 0.9800


In [43]:
metrics

{'model1': [0.06365036219358444, 0.9800000190734863]}